In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sysid as sd

## 1 Data analysis

### 1.2 Robot arm

In [ ]:
df = pd.read_csv("data/robot-arm.csv")
df

In [ ]:
t = np.arange(0, df.shape[0] * 0.1, 0.1)
u = df["u"].values
y = df["y"].values

In [ ]:
t = np.arange(0, 20, 0.1)
u = np.random.uniform(-1, 1, 200)
y = np.zeros_like(u)

for k in range(2, len(y)):
    y[k] = -0.605 * y[k - 1] - 0.163 * y[k - 2]**2 + 0.588 * u[k - 1] - 0.240 * u[k - 2]

In [ ]:
# separate the data into training and testing sets
n_train = 50 # int(0.2 * df.shape[0])

t_train, t_test = t[-n_train:], t[:-n_train]
u_train, u_test = u[-n_train:], u[:-n_train]
y_train, y_test = y[-n_train:], y[:-n_train]

In [ ]:
# Plot the train data
plt.plot(t_train, u_train, label="u")
plt.plot(t_train, y_train, label="y")
plt.xlabel("time (s)")
plt.title("Robot arm system")
plt.legend()
plt.show()

In [ ]:
nu, ny = 2, 2
dm = sd.data_matrix(u_train, y_train, nu, ny)
dm

In [ ]:
dm.shape

In [ ]:
l=3
cm, comb = sd.candidate_matrix(dm, l)
cm

In [ ]:
cm.shape

In [ ]:
theta = np.linalg.inv(cm.T @ cm) @ cm.T @ y_train[:-max(nu, ny)]
theta

In [ ]:
list(map(lambda c: sd.get_model_term(c, nu, ny), comb))

In [ ]:
y_hat = cm @ theta

In [ ]:
plt.plot(t_train[:-max(nu, ny)], y_train[:-max(nu, ny)], label="y")
plt.plot(t_train[:-max(nu, ny)], y_hat, label="y_hat")
plt.xlabel("time (s)")
plt.title("Robot Arm Train Set")
plt.legend()
plt.show()

In [ ]:
cm_test, _ = sd.candidate_matrix(sd.data_matrix(u_test, y_test, nu, ny), l)
y_hat_test = cm_test @ theta

In [ ]:
y_hat_test.shape

In [ ]:
# plot the test data
plt.plot(t_test[:-max(nu, ny)], y_test[:-max(nu, ny)], label="y")
plt.plot(t_test[:-max(nu, ny)], y_hat_test, label="y_hat")
plt.xlabel("time (s)")
plt.title("Robot Arm - Test set")
plt.legend()
plt.show()

In [ ]:
# plotting the abs error
plt.plot(t_test[:-max(nu, ny)], abs(y_test[:-max(nu, ny)] - y_hat_test), label="error")
plt.xlabel("time (s)")
plt.ylabel("absolute error")
plt.title("Robot Arm - Test set")
plt.legend()
plt.show()

In [ ]:
# computing mean absolute error
mae = np.mean(abs(y_test[:-max(nu, ny)] - y_hat_test))
mae

In [ ]:
cm.shape

# 2 Structure selection

## 2.1 Procedimento de Gram-Schmidt

In [ ]:
w1i = [cm[:, i] for i in range(cm.shape[1])]
w1i

In [ ]:
M = len(w1i)
M

In [ ]:
g1i = [w1i[i] @ y_train[:-max(nu, ny)] / (w1i[i] @ w1i[i]) for i in range(len(w1i))]
g1i

In [ ]:
ERRi = [g1i[i]**2 * w1i[i] @ w1i[i] / (y_train[:-max(nu, ny)] @ y_train[:-max(nu, ny)]) for i in range(len(w1i))]
ERRi

In [ ]:
selected = [np.argmax(ERRi)]
selected

In [ ]:
n_theta = 4

In [ ]:
W = [w1i[selected[0]]]
W

In [ ]:
for k in range(1, n_theta):
    ERRi = []
    for i in range(M):
        if i not in selected:
            alpha = [(W[j] @ w1i[i]) / (W[j] @ W[j]) for j in range(k)]
            wki = w1i[i] - sum([alpha[j] * W[j] for j in range(k)])
            gki = wki @ y_train[:-max(nu, ny)] / (wki @ wki)
            ERRi.append(gki**2 * wki @ wki / (y_train[:-max(nu, ny)] @ y_train[:-max(nu, ny)]))
        else:
            ERRi.append(0)
    selected.append(np.argmax(ERRi))
    W.append(w1i[selected[k]])

In [ ]:
selected

In [ ]:
list(map(lambda i: sd.get_model_term(comb[i], nu, ny), selected))

In [ ]:
selected_cm = cm[:, selected]
selected_cm

In [ ]:
selected_theta = np.linalg.inv(selected_cm.T @ selected_cm) @ selected_cm.T @ y_train[:-max(nu, ny)]
selected_theta

In [ ]:
y_hat_selected = selected_cm @ selected_theta
y_hat_selected

In [ ]:
# plot the selected train data
plt.plot(t_train[:-max(nu, ny)], y_train[:-max(nu, ny)], label="y")
plt.plot(t_train[:-max(nu, ny)], y_hat_selected, label="y_hat")
plt.xlabel("time (s)")
plt.title("Robot Arm Train Set")
plt.legend()
plt.show()

In [ ]:
y_hat_selected_test = cm_test[:, selected] @ selected_theta

In [ ]:
# plot the selected test data
plt.plot(t_test[:-max(nu, ny)], y_test[:-max(nu, ny)], label="y")
plt.plot(t_test[:-max(nu, ny)], y_hat_selected_test, label="y_hat")
plt.xlabel("time (s)")
plt.title("Robot Arm Test Set")
plt.legend()
plt.show()

In [ ]:
# plot the selected test error
plt.plot(t_test[:-max(nu, ny)], abs(y_test[:-max(nu, ny)] - y_hat_selected_test), label="error")
plt.xlabel("time (s)")
plt.ylabel("absolute error")
plt.title("Robot Arm Test Set")
plt.legend()
plt.show()

In [ ]:
# computing mean absolute error
np.mean(abs(y_test[:-max(nu, ny)] - y_hat_selected_test))